In [9]:
import pandas as pd
import numpy as np
import preprocessor as p
import re
# !pip install tweet-preprocessor

In [3]:
data1 = 'Data/archive-txt/'
data2 = 'Data/Tweets_Full_Data_data.xlsx'
data3 = 'Data/20200325_counsel_chat.csv'

In [8]:
counsel_df = pd.read_csv(data3)
counsel_df = counsel_df.drop(columns=['questionID','Unnamed: 0', 'questionLink', 'therapistInfo', 
                                      'therapistURL','upvotes','views','split'])


In [11]:
test = 'hello. im valeed. how are you! but im fine'
re.split('[?!.,]', test)

['hello', ' im valeed', ' how are you', ' but im fine']

In [66]:
topics_df = counsel_df.drop(columns=['answerText'])
topics_df = topics_df.drop_duplicates(subset=['questionTitle','questionText'])
topics_df['sentence'] = topics_df.questionTitle + ' ' + topics_df.questionText
topics_df = topics_df.drop(columns=['questionTitle','questionText'])
topics_df['sentence'] = topics_df.sentence.str.replace(':','')
topics_df['sentence'] = topics_df.sentence.str.replace('\n','')
topics_df['sentence'] = topics_df.sentence.str.replace('’','\'')

tmp = topics_df.topic.copy()
topics_df.topic = topics_df.sentence
topics_df.sentence = tmp
topics_df.columns = ['sentence','topic']


# for nlu2 
dataframes = []
for topic in topics_df.topic.unique():
    sentences = []
    sub_df = topics_df[topics_df.topic == topic]
    for sent in sub_df.sentence:
        sentences += re.split('[?!.]',sent)
    sentences = [s.strip() for s in sentences if s!='']
        
    d = {
            'sentence': sentences,
             'topic': [topic for i in range(len(sentences))]
        }
    dataframes.append(pd.DataFrame(data=d))

final = pd.concat(dataframes)
final.head()

,sentence,topic
0,Can I change my feeling of being worthless to ...,depression
1,I'm going through some things with my feelings...,depression
2,I barely sleep and I do nothing but think abou...,depression
3,I've never tried or contemplated suicide,depression
4,"I've always wanted to fix my issues, but I nev...",depression


In [67]:
intents = final.topic.unique()
intents = ["topic_"+ i for i in intents]

In [68]:
file = "nlu2.yml"
print(intents)
write_all_intents(file, intents, final)

['topic_depression', 'topic_anxiety', 'topic_parenting', 'topic_self-esteem', 'topic_relationship-dissolution', 'topic_workplace-relationships', 'topic_spirituality', 'topic_trauma', 'topic_domestic-violence', 'topic_anger-management', 'topic_sleep-improvement', 'topic_intimacy', 'topic_grief-and-loss', 'topic_substance-abuse', 'topic_family-conflict', 'topic_marriage', 'topic_eating-disorders', 'topic_relationships', 'topic_lgbtq', 'topic_behavioral-change', 'topic_addiction', 'topic_legal-regulatory', 'topic_professional-ethics', 'topic_stress', 'topic_human-sexuality', 'topic_social-relationships', 'topic_children-adolescents', 'topic_military-issues', 'topic_diagnosis', 'topic_counseling-fundamentals']


In [4]:

tweet_df = pd.read_excel(data2)
clean = [p.clean(t) for t in tweet_df.Tweet]
bad = ['(',')','-','{','}']
tweet_df.Tweet = [re.sub(r'[(){}\-\\]', '', t).strip().lower() for t in clean]
tweet_df = tweet_df.drop(['Id','Score (bin)', 'Testflag','Score'], axis=1)
tweet_df = tweet_df.rename(columns={"Tweet": "sentence", "Emotion": "emotion"})


In [5]:
train_df = pd.read_csv(data1+'train.txt', sep="\n", header=None)
train_df.columns = ["sent"]
train_df['sentence'] = train_df.sent.str.split(';').str[0]
train_df['emotion'] = train_df.sent.str.split(';').str[1]
train_df = train_df.drop(['sent'],axis=1)

In [6]:
test_df = pd.read_csv(data1+'test.txt', sep="\n", header=None)
test_df.columns = ["sent"]
test_df['sentence'] = test_df.sent.str.split(';').str[0]
test_df['emotion'] = test_df.sent.str.split(';').str[1]
test_df = test_df.drop(['sent'],axis=1)

In [7]:
val_df = pd.read_csv(data1+'val.txt', sep="\n", header=None)
val_df.columns = ["sent"]
val_df['sentence'] = val_df.sent.str.split(';').str[0]
val_df['emotion'] = val_df.sent.str.split(';').str[1]
val_df = val_df.drop(['sent'],axis=1)

In [8]:
val_df.sentence = val_df.sentence.str.lower()
intents = val_df.emotion.unique()
intents1 = ["mood_"+ i for i in intents]
intents2 =["mood_"+ i for i in tweet_df.emotion.unique()]
intents = np.unique(intents1+intents2)
intents

array(['mood_anger', 'mood_fear', 'mood_joy', 'mood_love', 'mood_sadness',
       'mood_surprise'], dtype='<U13')

In [13]:
data = pd.concat([train_df, test_df, val_df,tweet_df], ignore_index=True)
data = data.sample(frac=1)
data.head()

,sentence,emotion
10024,when my little sister was sick at home and i t...,fear
11159,i was feeling so rotten about it,sadness
20434,my momma irritate me asking all these question...,anger
15187,id have spent more time with her on reading i ...,sadness
5023,i feel glamorous rich enough for enriching my ...,joy


In [40]:
def write_intent(file, examples, intent):
    f = file
    f.write("- intent: "+ intent + "\n" + "  examples: |" + "\n")
    for e in examples:
        f.write("    - "+ e + "\n")
    

In [39]:
def write_all_intents(file_name, intents, df):
    f = open(file,"w")
    for i in intents:
        intent = i.split("_")[1]
        examples = [s for s in df[df.topic == intent].sentence]
        write_intent(f, examples, i)
    f.close()
    

In [12]:
file = "nlu.yml"
print(intents)
write_all_intents(file, intents, data)